1. Load the packages

In [ ]:
import random
import os
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta, Nadam
from keras import backend as K
import numpy   as np
import sys
import tensorflow as tf

2. Define the function used to load MNIST and USPS data splits



You need to download[ MNIST-USPS data splits](https://github.com/samotiian/CCSA) generated in [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put these splits in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put these splits in the corresponding folder of your device.



[1] Motiian, S., Piccirilli, M., Adjeroh, D. A., & Doretto, G. (2017). Unified deep supervised domain adaptation and generalization. In Proceedings of the IEEE international conference on computer vision (pp. 5715-5725).

In [ ]:
# Mount the Google drive, please ignore this cell if you run the code locally.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

initial_path='/content/drive/MyDrive/MINIST domain adaptation/CCSA-master/row_data/'

def printn(string):
    sys.stdout.write(string)
    sys.stdout.flush()

def Create_Pairs(domain_adaptation_task,repetition,sample_per_class):

    UM  = domain_adaptation_task
    cc  = repetition
    SpC = sample_per_class

    if UM != 'MNIST_to_USPS':
        if UM != 'USPS_to_MNIST':
            raise Exception('domain_adaptation_task should be either MNIST_to_USPS or USPS_to_MNIST')


    if cc <0 or cc>10:
        raise Exception('number of repetition should be between 0 and 9.')

    if SpC <1 or SpC>7:
            raise Exception('number of sample_per_class should be between 1 and 7.')


    print('Creating pairs for repetition: '+str(cc)+' and sample_per_class: '+str(sample_per_class))

    X_train_target=np.load(initial_path + UM + '_X_train_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    y_train_target=np.load(initial_path + UM + '_y_train_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')

    X_train_source=np.load(initial_path + UM + '_X_train_source_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    y_train_source=np.load(initial_path + UM + '_y_train_source_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')



    Training_P=[]
    Training_N=[]


    for trs in range(len(y_train_source)):
        for trt in range(len(y_train_target)):
            if y_train_source[trs]==y_train_target[trt]:
                Training_P.append([trs,trt])
            else:
                Training_N.append([trs,trt])


    random.shuffle(Training_N)
    Training = Training_P+Training_N[:3*len(Training_P)]
    random.shuffle(Training)


    X1=np.zeros([len(Training),16,16],dtype='float32')
    X2=np.zeros([len(Training),16,16],dtype='float32')

    y1=np.zeros([len(Training)])
    y2=np.zeros([len(Training)])
    yc=np.zeros([len(Training)])

    for i in range(len(Training)):
        in1,in2=Training[i]
        X1[i,:,:]=X_train_source[in1,:,:]
        X2[i,:,:]=X_train_target[in2,:,:]

        y1[i]=y_train_source[in1]
        y2[i]=y_train_target[in2]
        if y_train_source[in1]==y_train_target[in2]:
            yc[i]=1

    if not os.path.exists('./pairs'):
        os.makedirs('./pairs')

    np.save('./pairs/' + UM + '_X1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', X1)
    np.save('./pairs/' + UM + '_X2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', X2)

    np.save('./pairs/' + UM + '_y1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', y1)
    np.save('./pairs/' + UM + '_y2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', y2)
    np.save('./pairs/' + UM + '_yc_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', yc)




3. Define networks and functions for training

In [ ]:


def Create_Model():

    img_rows, img_cols = 16, 16
    # nb_filters = 32
    pool_size = (2, 2)
    kernel_size = (5, 5)
    input_shape = (img_rows, img_cols, 1)

    model = Sequential()
    model.add(Convolution2D(32, (kernel_size[0], kernel_size[1]),
                            padding ='same',
                            input_shape=input_shape))
    model.add(tf.keras.layers.PReLU())
    model.add(Convolution2D(32, (kernel_size[0], kernel_size[1]),
                            padding ='same',
                            input_shape=input_shape))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, (kernel_size[0], kernel_size[1]),
                            padding ='same',
                            input_shape=input_shape))
    model.add(tf.keras.layers.PReLU())
    model.add(Convolution2D(64, (kernel_size[0], kernel_size[1]),
                            padding ='same',
                            input_shape=input_shape))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(128, (kernel_size[0], kernel_size[1]),
                            padding ='same',
                            input_shape=input_shape))
    model.add(tf.keras.layers.PReLU())
    model.add(Convolution2D(128, (kernel_size[0], kernel_size[1]),
                            padding ='same',
                            input_shape=input_shape))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))



    model.add(Flatten())
    model.add(Dense(84))

    return model



def euclidean_distance(vects):
    eps = 1e-08
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), eps))



def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def training_the_model(model,domain_adaptation_task,repetition,sample_per_class):
    nb_classes=10
    UM = domain_adaptation_task
    cc = repetition
    SpC = sample_per_class

    if UM != 'MNIST_to_USPS':
        if UM != 'USPS_to_MNIST':
            raise Exception('domain_adaptation_task should be either MNIST_to_USPS or USPS_to_MNIST')

    if cc < 0 or cc > 10:
        raise Exception('number of repetition should be between 0 and 9.')

    if SpC < 1 or SpC > 7:
        raise Exception('number of sample_per_class should be between 1 and 7.')


    epoch = 100  # Epoch number
    batch_size = 256

    X_test = np.load(initial_path + UM + '_X_test_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC)+'.npy')
    y_test = np.load(initial_path + UM + '_y_test_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC)+'.npy')
    X_test = X_test.reshape(X_test.shape[0], 16, 16, 1)
    y_test = np_utils.to_categorical(y_test, nb_classes)


    X1 = np.load('./pairs/' + UM + '_X1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')
    X2 = np.load('./pairs/' + UM + '_X2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')



    X1 = X1.reshape(X1.shape[0], 16, 16, 1)
    X2 = X2.reshape(X2.shape[0], 16, 16, 1)

    y1 = np.load('./pairs/' + UM + '_y1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')
    y2 = np.load('./pairs/' + UM + '_y2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')
    yc = np.load('./pairs/' + UM + '_yc_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')

    y1 = np_utils.to_categorical(y1, nb_classes)
    y2 = np_utils.to_categorical(y2, nb_classes)

    print('Training the model - Epoch '+str(epoch))
    nn=batch_size
    # best_Acc = 0
    best_Acc = []

    for e in range(epoch):
        # if e % 10 == 0:
        #     printn(str(e) + '->')
        for i in range(int(len(y2) / nn)):
            loss = model.train_on_batch([X1[i * nn:(i + 1) * nn, :, :, :], X2[i * nn:(i + 1) * nn, :, :, :]],
                                        [y1[i * nn:(i + 1) * nn, :], yc[i * nn:(i + 1) * nn, ]])
            loss = model.train_on_batch([X2[i * nn:(i + 1) * nn, :, :, :], X1[i * nn:(i + 1) * nn, :, :, :]],
                                        [y2[i * nn:(i + 1) * nn, :], yc[i * nn:(i + 1) * nn, ]])

        Out = model.predict([X_test, X_test])
        Acc_v = np.argmax(Out[0], axis=1) - np.argmax(y_test, axis=1)
        Acc = (len(Acc_v) - np.count_nonzero(Acc_v) + .0000001) / len(Acc_v)
        print('Epoch:',e,"Loss:",loss,"testing acc:",Acc)
        
        print('========')

    return best_Acc

4. Example

The below is an example of N=2, i.e., two samples per class from the target domain, on the task of MNIST to USPS.

You may change the value of variables to get other experimental results.

Note that when N gets larger, the training time **quadratically** increases.


In [ ]:


# let's assume MNIST->USPS task.
domain_adaptation_task = 'MNIST_to_USPS'   # USPS_to_MNIST is also another option.

# let's run the experiments when 1 target sample per calss is available in training.
# you can run the experiments for sample_per_class=1, ... , 7.
sample_per_class = 2




opt = tf.keras.optimizers.Adadelta(learning_rate=0.025, rho=0.95, epsilon=1e-07)


print('Domain Adaptation Task: ' + domain_adaptation_task)
# let's create the positive and negative pairs using row data.
# pairs will be saved in ./pairs directory

for repetition in range(9,10):  # amend as 10 for all repetition
    # size of digits 16*16
    img_rows, img_cols = 16, 16
    input_shape = (img_rows, img_cols, 1)
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)


    # number of classes for digits classification
    nb_classes = 10

    # Loss = (1-alpha)Classification_Loss + (alpha)CSA
    alpha = 0.25
    
    # Creating embedding function. This corresponds to the function g in the paper.
    # You may need to change the network parameters.
    model1=Create_Model()

    # Having two streams. One for source and one for target.
    processed_a = model1(input_a)
    processed_b = model1(input_b)


    # Creating the prediction function. This corresponds to h in the paper.
    out1 = Dropout(0.5)(processed_a)
    out1 = Dense(nb_classes)(out1)
    out1 = Activation('softmax', name='classification')(out1)


    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape, name='CSA')(
    [processed_a, processed_b])
    model = Model(inputs=[input_a, input_b], outputs=[out1, distance])
    # model.summary()
    model.compile(loss={'classification': 'categorical_crossentropy', 'CSA': contrastive_loss},
              optimizer=opt,
              loss_weights={'classification': 1 - alpha, 'CSA': alpha})
    Create_Pairs(domain_adaptation_task,repetition,sample_per_class)
    Acc=training_the_model(model,domain_adaptation_task,repetition,sample_per_class)


    print('Best accuracy for {} target sample per class and repetition {} is {}.'.format(sample_per_class,repetition,Acc ))

Domain Adaptation Task: MNIST_to_USPS
Creating pairs for repetition: 9 and sample_per_class: 2
Training the model - Epoch 100
Epoch: 0 Loss: [1.6262743473052979, 2.0978236198425293, 0.21162639558315277] testing acc: 0.3005555556111111
Epoch: 1 Loss: [0.9305417537689209, 1.045890212059021, 0.5844966173171997] testing acc: 0.6627777778333334
Epoch: 2 Loss: [0.522926926612854, 0.45536211133003235, 0.7256214022636414] testing acc: 0.7638888889444445
Epoch: 3 Loss: [0.34322166442871094, 0.23956705629825592, 0.6541855335235596] testing acc: 0.8138888889444444
Epoch: 4 Loss: [0.2872052490711212, 0.18433813750743866, 0.5958065986633301] testing acc: 0.8300000000555556
Epoch: 5 Loss: [0.24634680151939392, 0.1431727111339569, 0.5558691024780273] testing acc: 0.8400000000555556
Epoch: 6 Loss: [0.22582417726516724, 0.13149316608905792, 0.508817195892334] testing acc: 0.8450000000555556
Epoch: 7 Loss: [0.20087526738643646, 0.11085709184408188, 0.4709298014640808] testing acc: 0.8472222222777778
Epo